In [ ]:
import os
import csv
import torch
torch.cuda_is_available = torch.cuda.is_available()

In [8]:
train_path = './Stable-Diffusion-Inpaint/data/train_data'
val_path = './Stable-Diffusion-Inpaintdata/data/val_data'

In [ ]:
path = './Stable-Diffusion-Inpaint/data/train_data'
csvName = 'train.csv'
tr_flist = os.listdir(path)
tr_flist.sort()
with open('./'+csvName, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['raw_path', 'fixed_path', 'mask_path', 'partition'])
    writer.writerows([[f, f, f.replace('.jpg', '_mask.jpg'), 'train'] for f in tr_flist])

val_flist = os.listdir(val_path)
val_flist.sort()
with open('./Stable-Diffusion-Inpaint/data/val.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['raw_path', 'fixed_path', 'mask_path', 'partition'])
    writer.writerows([[f, f.replace('.jpg', '_fixed.jpg'), f.replace('.jpg', '_mask.jpg'), 'validation'] for f in val_flist])

In [ ]:
import os
import glob

# 定義目錄
raw_dir = "./dataset_2025_03/train_data/origin/"
fixed_dir = "./dataset_2025_03/train_data/fixed/"
mask_dir = "./dataset_2025_03/train_data/mask/"
out_dir = "./dataset_2025_03/train_data/"
# 批量處理 raw
for file_path in glob.glob(os.path.join(raw_dir, "*.jpg")):
    base_name = os.path.basename(file_path)  # 取得原始檔名
    new_name = os.path.join(out_dir, base_name.replace(".jpg", "_raw.jpg"))
    os.rename(file_path, new_name)

# 批量處理 fixed
for file_path in glob.glob(os.path.join(fixed_dir, "*.jpg")):
    base_name = os.path.basename(file_path)
    new_name = os.path.join(out_dir, base_name.replace(".jpg", "_fixed.jpg"))
    os.rename(file_path, new_name)

# 批量處理 mask
for file_path in glob.glob(os.path.join(mask_dir, "*.jpg")):
    base_name = os.path.basename(file_path)
    new_name = os.path.join(out_dir, base_name.replace(".jpg", "_mask.jpg"))
    os.rename(file_path, new_name)

print("檔案重新命名完成！")

In [ ]:
import os
import glob

# 定義目錄
raw_dir = "./dataset_2025_03/train_data/origin/"
fixed_dir = "./dataset_2025_03/train_data/fixed/"
mask_dir = "./dataset_2025_03/train_data/mask/"
out_dir = "./dataset_2025_03/train_data/"
# 批量處理 raw
for file_path in glob.glob(os.path.join(raw_dir, "*.jpg")):
    base_name = os.path.basename(file_path)  # 取得原始檔名
    new_name = os.path.join(out_dir, base_name.replace(".jpg", "_raw.jpg"))
    os.rename(file_path, new_name)

# 批量處理 fixed
for file_path in glob.glob(os.path.join(fixed_dir, "*.jpg")):
    base_name = os.path.basename(file_path)
    new_name = os.path.join(out_dir, base_name.replace(".jpg", "_fixed.jpg"))
    os.rename(file_path, new_name)

# 批量處理 mask
for file_path in glob.glob(os.path.join(mask_dir, "*.jpg")):
    base_name = os.path.basename(file_path)
    new_name = os.path.join(out_dir, base_name.replace(".jpg", "_mask.jpg"))
    os.rename(file_path, new_name)

print("檔案重新命名完成！")


In [37]:
import loralib as lora
from ldm.util import instantiate_from_config
from omegaconf import OmegaConf
base = ".\configs\latent-diffusion\inpainting_example_overfit.yaml"
config = OmegaConf.load(base)

model = instantiate_from_config(config['model']) #wget -O models/ldm/inpainting_big/model_compvis.ckpt https://ommer-lab.com/files/latent-diffusion/inpainting_big.zi
#print(model.state_dict().keys())


LatentInpaintDiffusion: Running in eps-prediction mode
DiffusionWrapper has 400.19 M params.
making attention of type 'none' with 512 in_channels
Working with z of shape (1, 3, 64, 64) = 12288 dimensions.
making attention of type 'none' with 512 in_channels
Using first stage also as cond stage.
modifying key 'model.diffusion_model.input_blocks.0.0.weight' and keeping its original 7 (channels) dimensions only
modifying key 'model.diffusion_model.input_blocks.0.0.weight' and keeping its original 7 (channels) dimensions only
Restored from models/ldm/inpainting_big/model_compvis.ckpt with 340 missing and 564 unexpected keys
Missing Keys: ['model.diffusion_model.input_blocks.1.0.in_layers.2.lora_A', 'model.diffusion_model.input_blocks.1.0.in_layers.2.lora_B', 'model.diffusion_model.input_blocks.1.0.in_layers.2.conv.weight', 'model.diffusion_model.input_blocks.1.0.in_layers.2.conv.bias', 'model.diffusion_model.input_blocks.1.0.emb_layers.1.lora_A', 'model.diffusion_model.input_blocks.1.0.emb

In [36]:
#from torchsummary import summary
lora.mark_only_lora_as_trainable(model)
model.summarize()
from pytorch_lightning import Trainer
trainer = Trainer()
#trainer.save_checkpoint("model.ckpt")



  | Name              | Type             | Params
-------------------------------------------------------
0 | model             | DiffusionWrapper | 400 M 
1 | first_stage_model | VQModelInterface | 53.2 M
-------------------------------------------------------
12.9 M    Trainable params
440 M     Non-trainable params
453 M     Total params
1,813.651 Total estimated model params size (MB)
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [35]:
for name, param in model.named_parameters():
    #print(name, param.requires_grad)
    if "lora" in name:  # 只開啟 LoRA 層
        print(name, param.requires_grad)


model.diffusion_model.input_blocks.1.0.in_layers.2.lora_A True
model.diffusion_model.input_blocks.1.0.in_layers.2.lora_B True
model.diffusion_model.input_blocks.1.0.emb_layers.1.lora_A True
model.diffusion_model.input_blocks.1.0.emb_layers.1.lora_B True
model.diffusion_model.input_blocks.1.0.out_layers.3.lora_A True
model.diffusion_model.input_blocks.1.0.out_layers.3.lora_B True
model.diffusion_model.input_blocks.2.0.in_layers.2.lora_A True
model.diffusion_model.input_blocks.2.0.in_layers.2.lora_B True
model.diffusion_model.input_blocks.2.0.emb_layers.1.lora_A True
model.diffusion_model.input_blocks.2.0.emb_layers.1.lora_B True
model.diffusion_model.input_blocks.2.0.out_layers.3.lora_A True
model.diffusion_model.input_blocks.2.0.out_layers.3.lora_B True
model.diffusion_model.input_blocks.3.0.in_layers.2.lora_A True
model.diffusion_model.input_blocks.3.0.in_layers.2.lora_B True
model.diffusion_model.input_blocks.3.0.emb_layers.1.lora_A True
model.diffusion_model.input_blocks.3.0.emb_lay